<a href="https://colab.research.google.com/github/RaymondSalim/google-drive-scripts/blob/master/Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Execute this code first before anything else
### **DON'T FORGET TO PASTE YOUR FILE ID BELOW IF YOU ARE UPLOADING client_id.json FROM GOOGLE DRIVE**





In [ ]:
!gdown --id #PASTE YOUR ID HERE

In [ ]:
from __future__ import print_function


import io
import sys
import pathlib
import requests
import shutil
import os
from os import system
from pathlib import Path
from googleapiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from googleapiclient.http import MediaIoBaseDownload
from oauth2client.tools import argparser, run_flow

class googlescripts:
  def download(self, fId, fName, teamDrive):
    # Start
      if (teamDrive):
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="('" + fId + "' in parents)"
            ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
              corpora = "user",
              q="('" + fId + "' in parents)"
            ).execute().get('files', [])

      print("\n\n")
      for f in fileList:
        print(f['name'] + " " + f['id'])

        if f['mimeType'] == 'application/vnd.google-apps.folder':
            dir = os.path.join( os.getcwd() , f['name'])
            
            if not os.path.exists(dir):
              os.mkdir(dir)

            os.chdir(dir)
            print("\n\nFiles in " + f['name'])
            self.download(f['id'], f['name'], teamDrive)
            os.chdir(tempDir)
        
        else:

          a = {}
          a[f['name']] = f['id']

          file_id = f['id']
          local_filename = f['name']
          request = DRIVE.files().get_media(fileId=file_id)
          fh = io.FileIO(local_filename, 'wb')
          downloader = MediaIoBaseDownload(fh, request)
          done = False
          while done is False:
              status, done = downloader.next_chunk()
              print( "Download %d%%." % int(status.progress() * 100))
          print(local_filename + " downloaded " + str(pathlib.Path().absolute().resolve()))
    # End
  def renameTeamDrive(self, fName, removeName, folderId, wholeFolder):
      if wholeFolder:
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="(name contains '" + fName + "')"
            ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="(name contains '" + fName + "' AND '"+ folderId +"' in parents)"
            ).execute().get('files', [])

      for f in fileList:
        print(f['name'] + "  " + f['id'])

      for f in fileList:
        originalName = f['name']
        originalNameLower = f['name'].lower()
        firstIndex = originalNameLower.index(fName.lower())
        newName = originalName[0:firstIndex:] + removeName + originalName[firstIndex + len(fName) : len(originalName):]
        DRIVE.files().update(
            fileId = f['id'],
            supportsAllDrives = 'true',
            body = {
              "name": newName,
              "driveId": teamDriveId
            }
        ).execute()
        print(newName)

  def renameDrive(self, fName, removeName, folderId, wholeFolder):
      if wholeFolder:
          fileList = DRIVE.files().list(
          corpora = "user",
          q="(name contains '" + fName + "')"
        ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
          corpora = "user",
          q="(name contains '" + fName + "' AND '"+ folderId +"' in parents)"
        ).execute().get('files', [])

      for f in fileList:
        print(f['name'] + "  " + f['id'])

      for f in fileList:
        originalName = f['name']
        originalNameLower = f['name'].lower()
        firstIndex = originalNameLower.index(fName.lower())
        newName = originalName[0:firstIndex:] + removeName + originalName[firstIndex + len(fName) : len(originalName):]
        DRIVE.files().update(
            fileId = f['id'],
            body = {
                "name": newName
            }
        ).execute()
        print(newName)


sys.argv = ['--noauth_local_webserver']
args = argparser.parse_args()
args.noauth_local_webserver = True

os.chdir('/content')

initDirectory = os.getcwd()
googleclass = googlescripts()

SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, args)
DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))


system('clear') 


---

# Google Copy **FROM** TeamDrive Script



Download any files/folders as long as you have viewing access and copies it to your drive


Maximum files/folders size depends on google colab runtime

*   To increase storage size, Runtime > Change Runtime type > GPU


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
os.chdir('/content')


teamDriveId = input("Please enter source Drive ID\n")
folderId = input("Please enter source folder ID\n")
folderName = input("Please enter the folder name for the downloaded files \n")

tempDir = os.path.join( os.getcwd(), folderName)
if not (os.path.exists(tempDir)):
    os.mkdir(tempDir)
os.chdir(tempDir)


googleclass.download(folderId, folderName, True)

moveQ = input("Move the files?\n")
finalDest = input("Where do you want to move the files to? Please enter the directory. \n ex: \"/gdrive/Shared drives/MyTeamDrive\"\n")

finalDir = os.path.normpath(finalDest)

while not os.path.exists(finalDir):
  print("Directory doesn't exists")
  finalDest = input("Where do you want to move the files to? Please enter the directory. \n ex: \"/gdrive/Shared drives/MyTeamDrive\"\n")
  finalDir = os.path.normpath(finalDest)

if not os.path.exists(finalDir):
  os.mkdir(finalDir)

if moveQ[0].lower() == "y":
  print("Moving Files")
  shutil.move(tempDir, finalDir)
  print("Moving Completed")


os.chdir(initDirectory)



---

# Google Copy **TO** TeamDrive Script



Download any files/folders as long as you have viewing access and copies it to your drive


Maximum files/folders size depends on google colab runtime

*   To increase storage size, Runtime > Change Runtime type > GPU


In [ ]:
folderId = input("Please enter source folder ID\n")
folderName = input("Please enter the folder name for the downloaded files \n")

tempDir = os.path.join( os.getcwd(), folderName)
if not (os.path.exists(tempDir)):
    os.mkdir(tempDir)
os.chdir(tempDir)


googleclass.download(folderId, folderName, False)

moveQ = input("Move the files?\n")
finalDest = input("Where do you want to move the files to? Please enter the directory. \n ex: \"/gdrive/Shared drives/MyTeamDrive\"\n")

finalDir = os.path.normpath(finalDest)

while not os.path.exists(finalDir):
  print("Directory doesn't exists")
  finalDest = input("Where do you want to move the files to? Please enter the directory. \n ex: \"/gdrive/Shared drives/MyTeamDrive\"\n")
  finalDir = os.path.normpath(finalDest)

if not os.path.exists(finalDir):
  os.mkdir(finalDir)

if moveQ[0].lower() == "y":
  print("Moving Files")
  shutil.move(tempDir, finalDir)
  print("Moving Completed")


os.chdir(initDirectory)

---
# Google Drive Batch Rename Script

You can rename multiple files at once such as removing 'Copy of'

In [ ]:
SCOPES = 'https://www.googleapis.com/auth/drive'

isTeamDrive = input("Are the files located in a Team Drive/Shared Drives? (Yes / No)")

while not (isTeamDrive[0].lower() == "y" or isTeamDrive[0].lower() == "n"):
  isTeamDrive = input("Are the files located in a Team Drive/Shared Drives? (Yes / No)")

if (isTeamDrive[0].lower() == "y"):
  teamDriveId = input("Please enter TeamDrive ID\n")
  removeString = input("Please input the string you would like to replace")
  newString = input("What do you want to replace it with?")
  folder = input("What is the folder ID? (Ignore if you would like to search the whole drive)\n")
  wholeFolder = False
  if len(folder) == 0:
    wholeFolder = True
  googleclass.renameTeamDrive(removeString, newString, folder, wholeFolder)

elif (isTeamDrive[0].lower() == "n"):
  removeString = input("Please input the string you would like to replace\n")
  newString = input("What do you want to replace it with?\n")
  folder = input("What is the folder ID? (Ignore if you would like to search the whole drive)\n")
  wholeFolder = False
  if len(folder) == 0:
    wholeFolder = True
  googleclass.renameDrive(removeString, newString, folder, wholeFolder)









In [ ]:
os.chdir("/content")